In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
import cPickle as pickle

In [3]:
%matplotlib inline

In [4]:
pd.set_option('display.max_columns', 500)

####Clean_before_model

In [5]:
df = pd.read_pickle('df_1to1999_features.pkl')

In [6]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN


In [7]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error
1318,1998,273829,8100 Occidental Ave S ...,OCCIDENTAL AVE S BETWEEN S KENYON ST AND S SUL...,2014-05-12 07:00:00,2014-05-16 07:00:00,2014-05-16 07:00:00,4 days,4,47.527606,-122.315635,"S Sullivan St, Seattle, WA, USA",5.484420,6.607923,5.807149,5.836553,9.897122,7.842734,2,1,5,69,530330264001,1500000US530330264001,"Block Group 1, Census Tract 264, King County, ...",272800,10533
1319,1999,270118,710 East Roy Street ...,ROY ST BETWEEN AURORA AVE N AND DEXTER AVE N ...,2014-04-18 07:00:00,2014-04-19 07:00:00,2014-04-19 07:00:00,1 days,1,47.632366,-122.342277,"Dexter Ave N, Seattle, WA 98109, USA",1.868875,0.882158,1.564524,1.531412,2.554804,0.809788,2,2,4,47,530330067001,1500000US530330067001,"Block Group 1, Census Tract 67, King County, W...",352200,14323


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320 entries, 0 to 1319
Data columns (total 27 columns):
OBJECTID                  1320 non-null int64
WOKEY                     1320 non-null int64
LOCATION                  1320 non-null object
ADDRDESC                  1320 non-null object
INITDT_dt                 1320 non-null datetime64[ns]
FLDSTARTDT_dt             1320 non-null datetime64[ns]
FLDENDDT_dt               1320 non-null datetime64[ns]
DURATION                  1320 non-null timedelta64[ns]
DURATION_td               1320 non-null float64
latitude                  1320 non-null float64
longitude                 1320 non-null float64
address                   1320 non-null object
Seattle_dist              1320 non-null float64
Space_Needle_dist         1320 non-null float64
Pike_Place_dist           1320 non-null float64
Convention_Center_dist    1320 non-null float64
Woodland_Park_dist        1320 non-null float64
Queene_Anne_dist          1320 non-null float64
INIT_Qu

1. Strip time of day from datetime.
2. Remove NaNs.
3. Drop OBJECTID, WOKEY, GEO.id, location, address, ADDRDESC, GEOID, GEO.display-label

In [13]:
df.ix[0,'INITDT_dt']

Timestamp('2010-03-29 07:00:00')

In [18]:
df.ix[0,'INITDT_dt'].strftime('%Y-%m-%d')

'2010-03-29'

In [24]:
df.ix[0,'INITDT_dt'].strftime('%Y-%m-%d').to_datetime()

AttributeError: 'str' object has no attribute 'to_datetime'

In [23]:
df.ix[0,'INITDT_dt'].to_datetime()

datetime.datetime(2010, 3, 29, 7, 0)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1320 entries, 0 to 1319
Data columns (total 27 columns):
OBJECTID                  1320 non-null int64
WOKEY                     1320 non-null int64
LOCATION                  1320 non-null object
ADDRDESC                  1320 non-null object
INITDT_dt                 1320 non-null datetime64[ns]
FLDSTARTDT_dt             1320 non-null datetime64[ns]
FLDENDDT_dt               1320 non-null datetime64[ns]
DURATION                  1320 non-null timedelta64[ns]
DURATION_td               1320 non-null float64
latitude                  1320 non-null float64
longitude                 1320 non-null float64
address                   1320 non-null object
Seattle_dist              1320 non-null float64
Space_Needle_dist         1320 non-null float64
Pike_Place_dist           1320 non-null float64
Convention_Center_dist    1320 non-null float64
Woodland_Park_dist        1320 non-null float64
Queene_Anne_dist          1320 non-null float64
INIT_Qu

In [27]:
df[pd.isnull(df.neighborhood_label)].shape

(50, 27)

In [29]:
df[pd.isnull(df.neighborhood_label)]

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error
45,73,340491,S HUDSON ST BETWEEN UTAH AVE S AND 1ST AVE S ...,S HUDSON ST BETWEEN UTAH AVE S AND 1ST AVE S ...,2015-03-19 07:00:00,2015-03-24 07:00:00,2015-03-24 07:00:00,5 days 00:00:00,5,47.504919,-122.333648,"1st Avenue South, Seattle, WA, USA",6.998045,8.018738,7.251202,7.356061,11.342359,9.227181,1,3,3,NaN,NaN,NaN,NaN,NaN,NaN
58,96,344147,Between the Rainier Beach Lightrail station ML...,S HENDERSON ST BETWEEN RENTON AVE S AND 46TH A...,2015-04-11 07:00:00,2015-04-14 07:00:00,2015-04-14 07:00:00,3 days 00:00:00,3,47.496627,-122.276053,"46th Ave S, Seattle, WA, USA",8.011059,9.217857,8.403104,8.344274,12.418083,10.455491,2,2,4,NaN,NaN,NaN,NaN,NaN,NaN
79,127,332286,"There some potholes need to be filled, and som...",S THISTLE ST BETWEEN 39TH AVE S AND 42ND AVE S...,2015-02-13 08:00:00,2015-02-18 08:00:00,2015-02-18 08:00:00,5 days 00:00:00,5,47.500460,-122.280901,"42nd Ave S, Seattle, WA, USA",7.687691,8.888444,8.074185,8.023051,12.099407,10.126693,1,4,2,NaN,NaN,NaN,NaN,NaN,NaN
89,140,328646,"S Orcas St at 47th Ave S:\n\n- 4700 block, and...",S ORCAS ST BETWEEN 47TH AVE S AND 48TH AVE S ...,2015-01-27 08:00:00,2015-01-28 08:00:00,2015-01-28 08:00:00,1 days 00:00:00,1,47.505143,-122.273783,"48th Ave S, Seattle, WA, USA",7.495465,8.716740,7.901378,7.823483,11.889464,9.951672,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN
101,155,328654,"203 BOREN AVE S, SEATTLE, WA ...",BOREN AVE S BETWEEN 12TH (TURN RD) AVE S AND 1...,2015-01-27 08:00:00,2015-01-30 08:00:00,2015-01-30 08:00:00,3 days 00:00:00,3,47.480500,-122.318022,"12th Ave S, Seattle, WA, USA",8.709469,9.782192,8.998477,9.065009,13.097079,11.003383,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN
108,164,322076,14419 Greenwood Ave N ...,GREENWOOD AVE N BETWEEN N 144TH ST AND N 145TH...,2014-12-25 08:00:00,2015-01-08 08:00:00,2015-01-08 08:00:00,14 days 00:00:00,14,47.734186,-122.343984,"N 145th St, Seattle, WA 98133, USA",8.858898,7.857615,8.599319,8.503013,4.557635,6.713191,4,-6,12,NaN,NaN,NaN,NaN,NaN,NaN
179,280,314849,"14410 GREENWOOD AVE N ALPHA MONTESSORI, SEATTL...",GREENWOOD AVE N BETWEEN N 144TH ST AND N 145TH...,2014-11-25 08:00:00,2014-12-03 08:00:00,2014-12-03 08:00:00,8 days 00:00:00,8,47.734186,-122.343984,"N 145th St, Seattle, WA 98133, USA",8.858898,7.857615,8.599319,8.503013,4.557635,6.713191,4,-5,11,NaN,NaN,NaN,NaN,NaN,NaN
184,285,314970,14342 WALLINGFORD AVE N ...,WALLINGFORD AVE N BETWEEN N 143RD ST AND N 145...,2014-11-26 08:00:00,2014-12-04 08:00:00,2014-12-04 08:00:00,8 days 00:00:00,8,47.734186,-122.343984,"N 145th St, Seattle, WA 98133, USA",8.858898,7.857615,8.599319,8.503013,4.557635,6.713191,4,-5,11,NaN,NaN,NaN,NaN,NaN,NaN
205,323,360163,117 S WASHINGTON ST ...,S WASHINGTON ST BETWEEN OCCIDENTAL AVE S AND 2...,2015-06-29 07:00:00,2015-06-30 07:00:00,2015-06-30 07:00:00,1 days 00:00:00,1,47.426947,-122.333367,"2nd Ave S, Seattle, WA, USA",12.384409,13.392677,12.633540,12.742334,16.717316,14.589102,2,0,6,NaN,NaN,NaN,NaN,NaN,NaN
208,327,329289,A pothole is located in the north bound outer ...,GREENWOOD AVE N BETWEEN N 144TH ST AND N 145TH...,2015-01-30 08:00:00,2015-02-03 08:00:00,2015-02-03 08:00:00,4 days 00:00:00,4,47.734186,-122.343984,"N 145th St, Seattle, WA 98133, USA",8.858898,7.857615,8.599319,8.503013,4.557635,6.713191,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df[pd.isnull(df.Median_Value)]

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days 00:00:00,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...",3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,3,42,530330053022,1500000US530330053022,"Block Group 2, Census Tract 53.02, King County...",NaN,NaN
6,10,7211,Republican and Minor Ave N ...,MINOR AVE N AND REPUBLICAN ST ...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days 00:00:00,4,47.623131,-122.333003,"Republican St & Minor Ave N, Seattle, WA 98109...",1.169861,0.781462,1.021139,0.813571,3.281758,1.527051,2,2,4,49,530330073001,1500000US530330073001,"Block Group 1, Census Tract 73, King County, W...",NaN,NaN
10,15,6242,51ST S & S Ryan ...,HILLSIDE DR E AND LAKE WASHINGTON BLVD E ...,2010-04-22 07:00:00,2010-04-22 07:00:00,2010-04-26 07:00:00,4 days 00:00:00,4,47.622480,-122.284664,"Lake Washington Blvd E & Hillside Dr E, Seattl...",2.483533,3.021072,2.828254,2.326059,4.527405,3.573024,2,2,4,54,530330078001,1500000US530330078001,"Block Group 1, Census Tract 78, King County, W...",NaN,NaN
23,38,342639,"2102 22ND AVE S, SEATTLE, WA 98144 ...",22ND AVE S BETWEEN S HILL ST AND S WALKER ST ...,2015-04-01 07:00:00,2015-04-06 07:00:00,2015-04-06 07:00:00,5 days 00:00:00,5,47.583702,-122.314112,"S Walker St, Seattle, WA, USA",1.766965,3.027356,2.225497,2.081495,6.144999,4.240382,2,2,4,79,530330094004,1500000US530330094004,"Block Group 4, Census Tract 94, King County, W...",NaN,NaN
45,73,340491,S HUDSON ST BETWEEN UTAH AVE S AND 1ST AVE S ...,S HUDSON ST BETWEEN UTAH AVE S AND 1ST AVE S ...,2015-03-19 07:00:00,2015-03-24 07:00:00,2015-03-24 07:00:00,5 days 00:00:00,5,47.504919,-122.333648,"1st Avenue South, Seattle, WA, USA",6.998045,8.018738,7.251202,7.356061,11.342359,9.227181,1,3,3,NaN,NaN,NaN,NaN,NaN,NaN
58,96,344147,Between the Rainier Beach Lightrail station ML...,S HENDERSON ST BETWEEN RENTON AVE S AND 46TH A...,2015-04-11 07:00:00,2015-04-14 07:00:00,2015-04-14 07:00:00,3 days 00:00:00,3,47.496627,-122.276053,"46th Ave S, Seattle, WA, USA",8.011059,9.217857,8.403104,8.344274,12.418083,10.455491,2,2,4,NaN,NaN,NaN,NaN,NaN,NaN
67,111,343927,Stewart and Boren ...,STEWART ST BETWEEN BOREN AVE AND MINOR AVE ...,2015-04-09 07:00:00,2015-04-13 07:00:00,2015-04-13 07:00:00,4 days 00:00:00,4,47.612826,-122.326770,"Minor Ave, Seattle, WA, USA",0.519894,1.177662,0.751792,0.249940,4.048375,2.238507,2,2,4,63,530330084002,1500000US530330084002,"Block Group 2, Census Tract 84, King County, W...",NaN,NaN
74,119,332537,6TH AVE BETWEEN PIKE ST AND PINE ST ...,6TH AVE BETWEEN PIKE ST AND PINE ST ...,2015-02-17 08:00:00,2015-02-20 08:00:00,2015-02-20 08:00:00,3 days 00:00:00,3,47.612370,-122.334827,"Pine St, Seattle, WA, USA",0.444644,0.878418,0.389780,0.161885,3.978763,2.041349,1,4,2,62,530330082001,1500000US530330082001,"Block Group 1, Census Tract 82, King County, W...",NaN,NaN
79,127,332286,"There some potholes need to be filled, and som...",S THISTLE ST BETWEEN 39TH AVE S AND 42ND AVE S...,2015-02-13 08:00:00,2015-02-18 08:00:00,2015-02-18 08:00:00,5 days 00:00:00,5,47.500460,-122.280901,"42nd Ave S, Seattle, WA, USA",7.687691,8.888444,8.074185,8.023051,12.099407,10.126693,1,4,2,NaN,NaN,NaN,NaN,NaN,NaN
89,140,328646,"S Orcas St at 47th Ave S:\n\n- 4700 block, and...",S ORCAS ST BETWEEN 47TH AVE S AND 48TH AVE S ...,2015-01-27 08:00:00,2015-01-28 08:00:00,2015-01-28 08:00:00,1 days 00:00:00,1,47.505143,-122.273783,"48th Ave S, Seattle, WA, USA",7.495465,8.716740,7.901378,7.823483,11.889464,9.951672,1,5,1,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df.apply(lambda x: np.sum(pd.isnull(x)/float(1320)))

OBJECTID                  0.000000
WOKEY                     0.000000
LOCATION                  0.000000
ADDRDESC                  0.000000
INITDT_dt                 0.000000
FLDSTARTDT_dt             0.000000
FLDENDDT_dt               0.000000
DURATION                  0.000000
DURATION_td               0.000000
latitude                  0.000000
longitude                 0.000000
address                   0.000000
Seattle_dist              0.000000
Space_Needle_dist         0.000000
Pike_Place_dist           0.000000
Convention_Center_dist    0.000000
Woodland_Park_dist        0.000000
Queene_Anne_dist          0.000000
INIT_Quarter              0.000000
days_end_FY               0.000000
INIT_month                0.000000
neighborhood_label        0.037879
GEOID                     0.037879
GEO.id                    0.040152
GEO.display-label         0.040152
Median_Value              0.094697
Margin_of_Error           0.094697
dtype: float64

I'm getting a lot of NaNs, unfortunately.  Not much I can do about it.  At least check if there's anything special about those.

In [31]:
df['b_no_Median_Value'] = pd.isnull(df.Median_Value)
df['b_no_neighborhood_label'] = pd.isnull(df.neighborhood_label)

In [32]:
df.groupby('b_no_Median_Value').describe().T

b_no_Median_Value       False                                                  \
                        count                    mean                     std   
Convention_Center_dist   1195                4.183748                2.322072   
DURATION                 1195  5 days 02:02:24.602510  7 days 17:51:21.741990   
DURATION_td              1195                 5.07113                7.741584   
INIT_Quarter             1195                2.148117                1.145519   
INIT_month               1195                5.453556                3.618787   
Margin_of_Error          1195                72200.27                69581.76   
Median_Value             1195                422552.1                151182.7   
OBJECTID                 1195                990.7724                569.1657   
Pike_Place_dist          1195                4.277746                2.263327   
Queene_Anne_dist         1195                4.263067                2.048653   
Seattle_dist             1195                4.281244                2.389904   
Space_Needle_dist        1195                4.206452                2.094269   
WOKEY                    1195                204479.5                115447.4   
Woodland_Park_dist       1195                 4.61304                2.627334   
b_no_neighborhood_label  1195                       0                       0   
days_end_FY              1195               0.5464435                3.618787   
latitude                 1195                47.63198              0.06212967   
longitude                1195               -122.3329              0.03355882   
neighborhood_label       1195                79.48033                27.40536   

b_no_Median_Value                                                           \
                                     min              25%              50%   
Convention_Center_dist        0.07096169         1.934475         4.291005   
DURATION                 0 days 23:00:00  1 days 00:00:00  3 days 00:00:00   
DURATION_td                            0                1                3   
INIT_Quarter                           1                1                2   
INIT_month                             1                3                4   
Margin_of_Error                     7920            34980            50905   
Median_Value                      162800           318900           379800   
OBJECTID                               1              507              980   
Pike_Place_dist                0.1344163         2.305565         4.222965   
Queene_Anne_dist               0.1769855         2.633943         4.123668   
Seattle_dist                   0.1331334         2.137361         4.297578   
Space_Needle_dist              0.1713758         2.343887            4.297   
WOKEY                               1591            81921           226499   
Woodland_Park_dist             0.4342049         2.699879          3.86328   
b_no_neighborhood_label            False                0                0   
days_end_FY                           -6               -2                2   
latitude                        47.49927         47.57776         47.63059   
longitude                      -122.4134        -122.3614        -122.3294   
neighborhood_label                    27               57               81   

b_no_Median_Value                                          True   \
                                     75%               max count   
Convention_Center_dist          6.080232          8.586268   125   
DURATION                 5 days 00:00:00  74 days 00:00:00   125   
DURATION_td                            5                74   125   
INIT_Quarter                           3                 4   125   
INIT_month                             8                12   125   
Margin_of_Error                    81419            649096     0   
Median_Value                      482900            961800     0   
OBJECTID                          1467.5       

In [33]:
df.groupby('b_no_neighborhood_label').describe().T

b_no_neighborhood_label False                                                  \
                        count                    mean                     std   
Convention_Center_dist   1270                4.067331                2.319918   
DURATION                 1270  5 days 00:49:22.204724  7 days 13:51:57.561749   
DURATION_td              1270                5.020472                7.575492   
INIT_Quarter             1270                2.155906                1.144119   
INIT_month               1270                5.471654                3.612045   
Margin_of_Error          1195                72200.27                69581.76   
Median_Value             1195                422552.1                151182.7   
OBJECTID                 1270                986.4323                570.4433   
Pike_Place_dist          1270                4.172632                2.256617   
Queene_Anne_dist         1270                 4.16684                2.033101   
Seattle_dist             1270                 4.17276                2.380292   
Space_Needle_dist        1270                4.099937                2.091399   
WOKEY                    1270                204202.9                115602.9   
Woodland_Park_dist       1270                4.537947                2.580753   
b_no_Median_Value        1270              0.05905512               0.2358207   
days_end_FY              1270               0.5283465                3.612045   
latitude                 1270                 47.6321              0.06051499   
longitude                1270               -122.3318              0.03323926   
neighborhood_label       1270                78.32756                27.53162   

b_no_neighborhood_label                                                     \
                                     min              25%              50%   
Convention_Center_dist        0.07096169         1.902415         4.057406   
DURATION                 0 days 23:00:00  1 days 00:00:00  3 days 00:00:00   
DURATION_td                            0                1                3   
INIT_Quarter                           1                1                2   
INIT_month                             1                3                4   
Margin_of_Error                     7920            34980            50905   
Median_Value                      162800           318900           379800   
OBJECTID                               1            497.5              972   
Pike_Place_dist                0.1344163         2.223049         4.085213   
Queene_Anne_dist               0.1769855         2.589964         3.927013   
Seattle_dist                   0.1331334         2.098615         4.110454   
Space_Needle_dist              0.1713758         2.305509         4.088108   
WOKEY                               1591         80857.25           226287   
Woodland_Park_dist             0.4342049         2.675035         3.795869   
b_no_Median_Value                  False                0                0   
days_end_FY                           -6               -2                2   
latitude                        47.49927         47.58317         47.63055   
longitude                      -122.4134        -122.3583        -122.3279   
neighborhood_label                    27               55               80   

b_no_neighborhood_label                                    True   \
                                     75%               max count   
Convention_Center_dist          6.001171          8.586268    50   
DURATION                 5 days 00:00:00  74 days 00:00:00    50   
DURATION_td                            5                74    50   
INIT_Quarter                           3                 4    50   
INIT_month                             8                12    50   
Margin_of_Error                    81419            649096     0   
Median_Value                      482900            961800     0   
OBJECTID                          1466.5       

In [34]:
from scipy import stats
def ttest_by(vals, by):
    '''Compute a t-test on a column based on an indicator for which sample the values are in.'''
    vals1 = vals[by]
    vals2 = vals[-by]
    
    return stats.ttest_ind(vals1, vals2, equal_var='False')

In [35]:
ttest_by(df.DURATION_td, pd.isnull(df.neighborhood_label))

(-0.38573039720753949, 0.6997585971572523)

In [36]:
ttest_by(df.DURATION_td, pd.isnull(df.Median_Value))

(-0.98962499138830928, 0.32253909063218278)

At least there doesn't seem to be a significant difference between NaNs and non-NaNs.

In [38]:
df[df.b_no_Median_Value != True].shape

(1195, 29)

In [39]:
df_clean = df[df.b_no_Median_Value != True]

In [40]:
df_clean[pd.isnull(df.neighborhood_label)]

C:\Users\andersrmr\Anaconda\lib\site-packages\pandas\core\frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,b_no_Median_Value,b_no_neighborhood_label


Interesting that Median Value = NaN is a superset of neighborhood_label = NaN.

In [41]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 1319
Data columns (total 29 columns):
OBJECTID                   1195 non-null int64
WOKEY                      1195 non-null int64
LOCATION                   1195 non-null object
ADDRDESC                   1195 non-null object
INITDT_dt                  1195 non-null datetime64[ns]
FLDSTARTDT_dt              1195 non-null datetime64[ns]
FLDENDDT_dt                1195 non-null datetime64[ns]
DURATION                   1195 non-null timedelta64[ns]
DURATION_td                1195 non-null float64
latitude                   1195 non-null float64
longitude                  1195 non-null float64
address                    1195 non-null object
Seattle_dist               1195 non-null float64
Space_Needle_dist          1195 non-null float64
Pike_Place_dist            1195 non-null float64
Convention_Center_dist     1195 non-null float64
Woodland_Park_dist         1195 non-null float64
Queene_Anne_dist           1195 non-nu

In [49]:
df[np.isfinite(df['Median_Value'])]

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,INIT_month,neighborhood_label,GEOID,GEO.id,GEO.display-label,Median_Value,Margin_of_Error,b_no_Median_Value,b_no_neighborhood_label
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA",5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,3,93,530330114011,1500000US530330114011,"Block Group 1, Census Tract 114.01, King Count...",307500,53269,False,False
2,3,2649,32nd Ave NE and NE 135th ...,NE 135TH ST BETWEEN 32ND AVE NE AND LAKE CITY ...,2010-03-31 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,1 days,1,47.707446,-122.301566,"Lake City Way NE, Seattle, WA, USA",7.137526,6.405817,7.012778,6.783446,3.623119,5.502601,1,3,3,104,530330011001,1500000US530330011001,"Block Group 1, Census Tract 11, King County, W...",416900,62441,False,False
3,4,1591,Charles st & 6 Ave S pothole next to traffic l...,S CHARLES ST BETWEEN 6TH AVE S AND MAYNARD AVE...,2010-03-15 07:00:00,2010-03-16 07:00:00,2010-03-16 07:00:00,1 days,1,47.597085,-122.325065,"Maynard Ave S, Seattle, WA, USA",0.710334,1.974308,1.184267,1.035443,5.113765,3.183602,1,3,3,61,530330091002,1500000US530330091002,"Block Group 2, Census Tract 91, King County, W...",328900,145688,False,False
4,6,5642,52nd St. & 18th Ave. NE ...,18TH AVE NE BETWEEN NE 52ND ST AND NE 55TH ST ...,2010-04-19 07:00:00,2010-04-22 07:00:00,2010-04-22 07:00:00,3 days,3,47.668522,-122.295630,"NE 55th St, Seattle, WA 98105, USA",4.628992,4.156594,4.607589,4.291005,2.707355,3.612252,2,2,4,36,530330042005,1500000US530330042005,"Block Group 5, Census Tract 42, King County, W...",637000,56463,False,False
5,7,7585,...,3RD AVE NW BETWEEN NW 125TH ST AND NW 127TH ST...,2010-05-03 07:00:00,2010-05-04 07:00:00,2010-05-04 07:00:00,1 days,1,47.721476,-122.364984,"NW 127th St, Seattle, WA 98177, USA",8.110129,7.014027,7.793727,7.762688,3.695250,5.813108,2,1,5,110,530330005002,1500000US530330005002,"Block Group 2, Census Tract 5, King County, Wa...",565200,53534,False,False
7,11,3168,...,17TH AVE W BETWEEN W DRAVUS ST AND W BERTONA S...,2010-04-02 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,4 days,4,47.650311,-122.388604,"W Bertona St, Seattle, WA, USA",4.031574,2.758997,3.544222,3.781162,2.061169,1.686262,2,2,4,50,530330058012,1500000US530330058012,"Block Group 2, Census Tract 58.01, King County...",255200,50905,False,False
8,13,6967,B/T Pike and Pine B/t 211 and 215 ...,PIKE ST BETWEEN 2ND AVE AND 3RD AVE ...,2010-04-28 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,5 days,5,47.610741,-122.338703,"3rd Ave, Seattle, WA, USA",0.440475,0.836150,0.177650,0.331137,4.053555,2.052527,2,2,4,62,530330081002,1500000US530330081002,"Block Group 2, Census Tract 81, King County, W...",384400,224247,False,False
9,14,7107,17th Ave NE Exit of University of Washington ...,NE 45TH ST BETWEEN 15TH AVE NE AND 16TH AVE NE...,2010-04-29 07:00:00,2010-05-03 07:00:00,2010-05-03 07:00:00,4 days,4,47.675800,-122.310849,"16th Ave NE, Seattle, WA, USA",4.908781,4.220325,4.794037,4.554949,2.059034,3.439953,2,2,4,36,530330026003,1500000US530330026003,"Block Group 3, Census Tract 26, King County, W...",450600,59449,False,False
11,16,5239,3708 2nd Ave S ...,2ND AVE S BETWEEN S SPOKANE (S RD) ST AND S AN...,2010-04-15 07:00:00,2010-04-16 07:00:00,2010-04-16 07:00:00,1 days,1,47.568293,-122.301841,"S Andover St, Seattle, WA, USA",2.976221,4.233906,3.427501,3.287720,7.336160,5.449602,2,2,4,79,530330100011,1500000US530330100011,"Block Group 1, Census Tract 100.01, King Count...",358200,39966,False,False
12,17,4589,112 NE 92nd ...,NE 92ND ST BETWEEN 1ST AVE NE AND 2ND AVE NE ...,2010-04-12 07:00:00,2010-04-14 07:00:00,2010-04-14 07:00

In [52]:
np.any(pd.isnull(df.Median_Value))

True

In [53]:
df.isnull().values.any()

True

In [54]:
df_clean.isnull().values.any()

False

In [55]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 1319
Data columns (total 29 columns):
OBJECTID                   1195 non-null int64
WOKEY                      1195 non-null int64
LOCATION                   1195 non-null object
ADDRDESC                   1195 non-null object
INITDT_dt                  1195 non-null datetime64[ns]
FLDSTARTDT_dt              1195 non-null datetime64[ns]
FLDENDDT_dt                1195 non-null datetime64[ns]
DURATION                   1195 non-null timedelta64[ns]
DURATION_td                1195 non-null float64
latitude                   1195 non-null float64
longitude                  1195 non-null float64
address                    1195 non-null object
Seattle_dist               1195 non-null float64
Space_Needle_dist          1195 non-null float64
Pike_Place_dist            1195 non-null float64
Convention_Center_dist     1195 non-null float64
Woodland_Park_dist         1195 non-null float64
Queene_Anne_dist           1195 non-nu

In [56]:
df_all = df_clean.ix[:, ['neighborhood_label','INIT_Quarter','INIT_month','days_end_FY', 'Median_Value',
                                        'Queene_Anne_dist','Woodland_Park_dist','Space_Needle_dist',
                                       'Seattle_dist','latitude','longitude','INITDT_dt','FLDENDDT_dt',
                                       'Convention_Center_dist','Pike_Place_dist','Margin_of_Error']]

In [57]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 1319
Data columns (total 16 columns):
neighborhood_label        1195 non-null float64
INIT_Quarter              1195 non-null int64
INIT_month                1195 non-null int64
days_end_FY               1195 non-null int64
Median_Value              1195 non-null float64
Queene_Anne_dist          1195 non-null float64
Woodland_Park_dist        1195 non-null float64
Space_Needle_dist         1195 non-null float64
Seattle_dist              1195 non-null float64
latitude                  1195 non-null float64
longitude                 1195 non-null float64
INITDT_dt                 1195 non-null datetime64[ns]
FLDENDDT_dt               1195 non-null datetime64[ns]
Convention_Center_dist    1195 non-null float64
Pike_Place_dist           1195 non-null float64
Margin_of_Error           1195 non-null float64
dtypes: datetime64[ns](2), float64(11), int64(3)
memory usage: 158.7 KB


In [62]:
df_data = pd.concat([df_all.ix[:, ['Queene_Anne_dist','Woodland_Park_dist',
                    'Space_Needle_dist','Convention_Center_dist','Pike_Place_dist',
                    'Seattle_dist','latitude','longitude','Median_Value','Margin_of_Error']],
                    df_all.neighborhood_label.astype('category'), 
                    df_all.INIT_Quarter.astype('category'), 
                    df_all.INIT_month.astype('int')], axis=1)

In [63]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1195 entries, 0 to 1319
Data columns (total 13 columns):
Queene_Anne_dist          1195 non-null float64
Woodland_Park_dist        1195 non-null float64
Space_Needle_dist         1195 non-null float64
Convention_Center_dist    1195 non-null float64
Pike_Place_dist           1195 non-null float64
Seattle_dist              1195 non-null float64
latitude                  1195 non-null float64
longitude                 1195 non-null float64
Median_Value              1195 non-null float64
Margin_of_Error           1195 non-null float64
neighborhood_label        1195 non-null category
INIT_Quarter              1195 non-null category
INIT_month                1195 non-null int32
dtypes: category(2), float64(10), int32(1)
memory usage: 110.4 KB


In [64]:
y = df_clean['DURATION_td']
X = df_data